In [118]:
from llama_index.core import VectorStoreIndex,  Settings
from llama_index.llms.together import TogetherLLM
from llama_index.core.agent import ReActAgent
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

In [119]:
Settings.llm = TogetherLLM(
    model="meta-llama/Meta-Llama-3-70B-Instruct-Turbo", api_key="6f81d9a1dc6d93c9105d5827b3fc9c9717d24e462fc04c74369d49ab85dc03b6"
)


In [120]:
flight_data = {
        "meta": {
            "count": 2,
            "links": {
                "self": "https://test.api.amadeus.com/v2/shopping/flight-offers?originLocationCode=SYD&destinationLocationCode=BKK&departureDate=2021-11-01&adults=1&max=2"
            }
        },
        "data": [
            {
                "type": "flight-offer",
                "id": "1",
                "source": "GDS",
                "instantTicketingRequired": False,
                "nonHomogeneous": False,
                "oneWay": False,
                "lastTicketingDate": "2021-11-01",
                "numberOfBookableSeats": 9,
                # ... (other flight details)
            },
            # ... (second flight offer)
        ],
        "dictionaries": {
            "locations": {
                "BKK": {"cityCode": "BKK", "countryCode": "TH"},
                "MNL": {"cityCode": "MNL", "countryCode": "PH"},
                "SYD": {"cityCode": "SYD", "countryCode": "AU"}
            },
            "aircraft": {
                "320": "AIRBUSA320",
                "321": "AIRBUSA321",
                "333": "AIRBUSA330-300"
            },
            "currencies": {"EUR": "EURO"},
            "carriers": {"PR": "PHILIPPINEAIRLINES"}
        }
    }

In [121]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate
import json

def flight_offers(departure,arrival,departure_date,number_of_adults,travel_class):
   print("Here are some flight offers.")

def hotel_offers(city):
    #call am
    print("Here are some hotel offers.")
    
information_extract_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            """You are a friendly travel agent. You specialize in booking flights and hotels. Interact in a conversational and casual tone.
               The very first step is to determine what services that user would like to use.
               After determining what services the user wants to use, first write out that information in a list form. 
               Now, your job is to go through the list of services you made and ask questions. Strictly follow the steps presented below.
               
               For booking flights strictly just do the following steps:
               Step 1. Ask about city of departure (city only).
               
               Step 2. Ask about city of arrival (city only).
               
               Step 3. Ask about the departure date (YYYY-MM-DD).
               
               Step 4. Ask about the number of adult travelers (age 12 or older on date of departure).
               
               Step 5. Ask about the travel class (economy, premium economy, business, first).
               
               Step 6. Ask user for confirmation.
               
               Step 7. Go to the next service on your initial list. If there are no more, summarize everything and just output a singular <DONE>.
               
               For booking hotels strictly just do the following steps:
               Step 1. If the user used the booking flights service, infer the city that they would want to stay in. If they did not, then ask the city they would want to stay in.
               
               Step 2. Ask user for confirmation.
               
               Step 3. Go to the next service on your initial list. If there are no more, summarize everything just output a singular <DONE>.
               
               Here is the chat history to help you:
               {history}
            """
        ),
    ),
    ChatMessage(
        role=MessageRole.USER,
        content=(
            """
            {question}
            """
        ),
    ),


]
information_extract_template = ChatPromptTemplate(information_extract_msgs)



tools="""
1.flight_offers(departure=str,arrival=str,departure_date=str,number_of_adults=int,travel_class=(one of "ECONOMY", "PREMIUM_ECONOMY", "BUSINESS", "FIRST"))\n
2.hotel_offers(city=str)\n
"""
function_call_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            """You are a travel ai agent. Your job is to determine which functions are appropriate given a summary of 
            travel requirements from a user.\n
            Questions may involve the following topics:\n
            Booking flights\n
            Booking hotels\n
            """
        ),
    ),
    ChatMessage(role=MessageRole.USER, content="""
                Summary:
                
                {summary}
                
                Which function calls are most appropriate? (only select from the following functions):\n
                {tools}
                Answer format:[just the names of most appropriate functions with appropriate arguments in Python]
                """),
]
function_call_template = ChatPromptTemplate(function_call_msgs)


In [125]:
def print_filtered_string(original_string, substrings_to_ignore):
    for substring in substrings_to_ignore:
        original_string = original_string.replace(substring, "")
    print(original_string,end="")

substrings_to_ignore=["<D","ON","E",">","<DONE>"]
user_input=""
chat_history=""
last_response=""
while user_input!="exit":
    user_input=lower(input("Type your query here:"))
    response=Settings.llm.stream_complete(information_extract_template.format(question=user_input,history=chat_history))
    full_response=""
    for r in response:
        full_response+=r.delta
        print_filtered_string(r.delta,substrings_to_ignore)
    chat_history+=f"User:{user_input}\n You:{full_response}"
    if("<DONE>" in full_response):
        last_response=full_response.replace("<DONE>","")
        break


    

I'd be happy to help you with that.

So, you'd like to book a flight. Here's what I've got so far:

**Services:**

* Book a flight

Let's get started! For your flight, can you please tell me what city you'll be departing from?So you'll be departing from Sydney. Got it!

Next, can you please tell me what city you'll be flying to?Yangon! Got it!

Next, can you please tell me what date you'd like to depart? Please format it as YYYY-MM-DD.So you'd like to depart on September 4th, 2024. Got it!

Next, how many adult travelers (age 12 or older on the date of departure) will be flying?So there will be 2 adult travelers. Got it!

Next, what travel class would you prefer? We have economy, premium economy, business, or first class.So you'd like to fly economy class. Got it!

Just to confirm, I've got the following information for your flight:

**Services:**

* Book a flight

**Flight Details:**

* Departing from: Sydney
* Flying to: Yangon
* Departure date: 2024-09-04
* Number of adult travelers

In [126]:
response=Settings.llm.stream_complete(function_call_template.format(summary=last_response,tools=tools))
full_response=""
for r in response:
  full_response+=r.delta
  print(r.delta,end="")
print("\n")
exec("json_response="+full_response)

flight_offers(departure="Sydney",arrival="Yangon",departure_date="2024-09-04",number_of_adults=2,travel_class="ECONOMY")

Here are some flight offers.
